In [50]:
import numpy as np
import pandas as pd
import joblib

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

import joblib

In [107]:
df = pd.read_csv('training_set_7_final.txt', sep='\t', error_bad_lines=False, encoding="ISO-8859-1")

c:\users\20167947\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
df

,essay,domain1_score
0,Patience is when your waiting .I was patience ...,15
1,"I am not a patience person, like I cant sit i...",13
2,One day I was at basketball practice and I was...,15
3,I going to write about a time when I went to t...,17
4,It can be very hard for somebody to be patient...,13
...,...,...
1564,One time I was getting a cool @CAPS1 game it w...,12
1565,A patent person in my life is my mom. Aicason ...,16
1566,A time when someone else I know was patient wa...,19
1567,I hate weddings. I love when people get marrie...,22


In [106]:
df_neg = pd.read_csv('essay_samples.csv', encoding = "ISO-8859-1")

In [54]:
def text_to_cleantext_tokenizer(text, remove_nonletters = False, remove_stopwords=False, stemming=False, lemma=False):

    # 1. Remove HTML
    teks = BeautifulSoup(text, 'lxml').get_text()

    # 2. Remove non-ASCII
    letters_only = re.sub(r"[^\x00-\x7F]+", " ", teks)

    if remove_nonletters:
        letters_only = re.sub("[^a-zA-Z]", " ", letters_only)

    #letters_only = teks

    # 3. Convert to lower-case, split into words
    words = letters_only.lower()
    words = word_tokenize(words)

    # 4. Convert stopwords into Set (faster than List)
    # 5. Remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    # 6. Stemming
    if stemming:
        porter = PorterStemmer()
        stems = []
        for t in words:
            stems.append(porter.stem(t))
        words = stems

    # 7. Stemming
    if lemma:
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for t in words:
            lemmas.append(lemmatizer.lemmatize(t))
        words = lemmas
        
    # 8. Join words back into one string by space, and return the result
    return(" ".join(words))
    # 8. Return list of words
    # return(words)
    

In [55]:
original_essays = df['essay']

In [56]:
negation_essays = df_neg['new_meaning']

In [72]:
all_essays = pd.concat([original_essays, negation_essays], ignore_index=True)
all_essays

0       Patience is when your waiting .I was patience ...
1       I am not a patience person, like I cant sit i...
2       One day I was at basketball practice and I was...
3       I going to write about a time when I went to t...
4       It can be very hard for somebody to be patient...
                              ...                        
1614    We climbed in the rental car saying hello to r...
1615    A short wait   @CAPS1 'only four' at the time....
1616    I had another math home work assignment due to...
1617    Once upon a time thime was a boy named @PERSON...
1618    This is a story about when I was impatient. My...
Length: 1619, dtype: object

In [75]:
df = pd.DataFrame(all_essays, columns=['essay'])
df

,essay
0,Patience is when your waiting .I was patience ...
1,"I am not a patience person, like I cant sit i..."
2,One day I was at basketball practice and I was...
3,I going to write about a time when I went to t...
4,It can be very hard for somebody to be patient...
...,...
1614,We climbed in the rental car saying hello to r...
1615,A short wait @CAPS1 'only four' at the time....
1616,I had another math home work assignment due to...
1617,Once upon a time thime was a boy named @PERSON...


In [78]:
df['essay'][0]

'Patience is when your waiting .I was patience when in line waiting for lunch .I didn\x92t c ut any one to eat .I was standing and waiting for my turn .Patience ,some people don\x92t have it .Lots of people just cut or yell at you because they don\x92t have  any patience. Sometimes people will push you out of their way .They only do that because they don\x92t have patience at all. Patience is what people need .People need patience because lots o f feelings get hurt .Everyone should have patience.'

In [80]:
for idx, row in df.iterrows():  
    line = df.at[idx, 'essay']
    line = line.encode('ascii','ignore')
    line = text_to_cleantext_tokenizer(line, remove_nonletters=True, remove_stopwords=False, lemma=True)
    df.at[idx,'essay'] = line

In [84]:
df['essay'][0]

'patience is when your waiting i wa patience when in line waiting for lunch i didnt c ut any one to eat i wa standing and waiting for my turn patience some people dont have it lot of people just cut or yell at you because they dont have any patience sometimes people will push you out of their way they only do that because they dont have patience at all patience is what people need people need patience because lot o f feeling get hurt everyone should have patience'

In [85]:
vectorizer = CountVectorizer(min_df=3, ngram_range=(1,2))
X = vectorizer.fit_transform(df['essay'])

In [86]:
len(vectorizer.get_feature_names_out())

15597

In [87]:
vectorizer.get_feature_names_out()

array(['ability', 'ability to', 'able', ..., 'zoo', 'zoom', 'zoomed'],
      dtype=object)

In [93]:
feats = X.toarray()

In [95]:
len(feats)

1619

In [101]:
neg = feats[-50:]

In [130]:
ori = feats[:1569]

In [112]:
idx = df_neg['index'].to_numpy()

In [131]:
ori = ori[idx]

In [113]:
type(idx)

numpy.ndarray

In [114]:
idx

array([ 582,  466, 1138,  178,   46, 1311,  353,  402,  675,  507,  973,
        267, 1263,  747, 1250, 1315,  205, 1356,   51,    0,  277,  660,
        412,  808, 1087, 1357,  171,  722, 1211, 1014,  903,  505,  259,
        355, 1312,  866,  291, 1195, 1177, 1106,  478, 1076,  386,  673,
       1241,  614, 1021, 1182, 1079,  131], dtype=int64)

In [132]:
len(ori)

50

In [133]:
len(neg)

50

In [134]:
joblib.dump(ori, 'ori_ngram')
joblib.dump(neg, 'neg_ngram')

['neg_ngram']

In [27]:
#joblib.dump(X.toarray(), 'essay_ngram_no_stopword_asap7')

['essay_ngram_no_stopword_asap7']